In [8]:
import random
import json
import re  

In [6]:
label_map = {} 
with open('/scratch/aditya/useful/subject_set_labels.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        label_map.update(line)

In [42]:
prop_dict = {} 
with open('/scratch/aditya/useful/ontology_props.jsonl', 'r') as f:
    for line in f:
        line = json.loads(line)
        prop_dict.update({line['name']: line['properties']})

In [23]:
def get_prop_name(item):
    if("#literal" in item):
        return item.split('#literal')[0], True
    if(item in label_map):
        tail_name = label_map[item]
    else:
        tail_name = re.sub('_', ' ',item.split('/')[-1])
    return tail_name, False

In [56]:
out_file = open('/scratch/aditya/props_corpus.txt', 'w')
with open('/scratch/aditya/useful/ontology_props.jsonl', 'r') as f:
    for i, line in enumerate(f):
        if(i%600000 == 0):
            print(i)
        tails = set() 
        out_props = []
        line = json.loads(line)
        resource_name = label_map[line['name']]
        forward_properties = line['properties']['properties']
        for prop in forward_properties:
            if(prop == 'name'):
                continue
            if(len(forward_properties[prop]) > 3):
                picked_rng = random.randint(1, 3)
                picked = random.sample(forward_properties[prop], picked_rng)
            else:
                picked = forward_properties[prop]
            for item in picked:
                tail_name, literal = get_prop_name(item)
                out_props.append(f'{resource_name} | {prop} | {tail_name}')

                if(not(literal)):
                    tails.add(tail_name)
                    chain_rng = random.randint(1, 20)
                    if(chain_rng < 2):
                        if(item in prop_dict):
                            forward_props = prop_dict[item]['properties']
                            forward_props_keys = list(prop_dict[item]['properties'].keys())
                            available_props = len(forward_props_keys)
                            if(available_props < 1):
                                continue 
                            picked_rng = random.randint(1, min(3, available_props))
                            picked_props = random.sample(forward_props_keys, picked_rng)
                            for prop_ in picked_props:
                                if(len(forward_props[prop_]) > 3):
                                    picked_rng_ = random.randint(1, 3)
                                    picked_ = random.sample(forward_props[prop_], picked_rng_)
                                else:
                                    picked_ = forward_props[prop_]

                                for item_ in picked_:
                                    tail_name_, literal = get_prop_name(item_)
                                    if(tail_name_ == resource_name):
                                        continue 
                                    out_props.append(f'{tail_name} | {prop_} | {tail_name_}')
        
        reverse_properties = line['properties']['reverse_properties']
        for prop in reverse_properties:
            if(len(reverse_properties[prop]) > 3):
                picked_rng = random.randint(1, 3)
                picked = random.sample(reverse_properties[prop], picked_rng)
            else:
                picked = reverse_properties[prop]
            for item in picked:
                tail_name, literal = get_prop_name(item)
                if(tail_name in tails):
                    continue
                tails.add(tail_name)
                out_props.append(f'{tail_name} | {prop} | {resource_name}')

        random_order = random.randint(1, 10)
        if(random_order < 5):
            random.shuffle(out_props)
        out_props = [i.strip() for i in out_props if len(i.strip()) > 0]
        if(len(out_props) > 0):
            joined_props = ' <TSP> '.join(out_props)
            out_file.write(f'{joined_props}\n')
out_file.close()

0
600000
1200000
1800000
2400000
3000000
